# PBT練習帳

## 事例ベーステスト vs プロパティベーステスト

プロパティベーステストでは個別の入出力ではなく守られるべきルールを記述する。

> 従来のテストの多くは事例ベースです。ここで言う事例とは、プログラムへの入力値と期待される出力値とを組にして並べたものです。プログラムが行うべき事柄について数行のコメントを書くこともあるかもしれませんが、それ以上のものではありません。優れたテストであるかどうかは、「プログラムにおいて取りうるすべての状態をあなたが用意した事例で試せるかどうか」で決まります。
>
> 対照的に、プロパティベーステストは手で事例を並べる作業とは無縁です。代わりに、ある種のメタテストを書くことになります。つまり、「どのような入力値を与えても常に同じであるような振る舞い」を記述するルールを見つけて、それを実行可能なコードとして書き表すのです。その実行可能なコードが**プロパティ**です。すると、特別なテストフレームワークがテストケースを生成し、それらをプロパティに対して実行して、あなたが想定したルールがプログラム内で守られているかを確認します。
>
> <cite>[実践プロパティベーステスト &mdash; PropErとErlang/Elixrではじめよう](https://www.lambdanote.com/collections/proper-erlang-elixir), pp.7-8</cite>

- 事例ベーステスト
  - $x_k$ と $y_k$ の組を挙げて $f(x_k) = y_k$ を示す
  - $f(x_1) = y_1$
  - $f(x_2) = y_2$
  - $f(x_3) = y_3$
  - ...
- プロパティベーステスト
  - $f(x)$ に関する性質 $\mathrm{prop}(x)$ を定義して $\forall x\; \mathrm{prop}(x)$ を示す
  - $\mathrm{prop}_{\text{roundtrip}}(x) \mathrel{\overset{\text{def}}{\iff}} f^{-1}(f(x)) = x$
    - $f = \text{write}$, $f^{-1} = \text{read}$\
      書いたものを読んだら元の値が得られる
  - $\mathrm{prop}_{\text{idempotency}}(x) \mathrel{\overset{\text{def}}{\iff}} f(f(x)) = f(x)$
    - $f = \text{PUT}$\
      2回PUTしても結果は同じ
  - $\mathrm{prop}_{\text{invariant}}(x) \mathrel{\overset{\text{def}}{\iff}} i(f(x)) = i(x)$
    - $f = \text{sort}$, $i = \text{length}$\
      ソートしても長さは変わらない
  - $\mathrm{prop}_{\text{oracle}}(x) \mathrel{\overset{\text{def}}{\iff}} f(x) = o(x)$
    - $f = \text{並列処理による実装}, o = \text{シーケンシャルでナイーブな実装}$\
      並列処理による実装もナイーブな実装と同等の出力を得られる
  - ...

## 準備

[hypothesis](https://github.com/HypothesisWorks/hypothesis)を用いる。

In [ ]:
%pip install hypothesis ipytest

In [ ]:
import ipytest
import pytest

ipytest.autoconfig()

## 「[プロパティベース・テストを習得する](https://blogs.oracle.com/otnjp/post/know-for-sure-with-property-based-testing-ja)」より

[プロパティベース・テストを習得する](https://blogs.oracle.com/otnjp/post/know-for-sure-with-property-based-testing-ja)を参考に、下記の簡単なクラスについて事例ベーステストとプロパティベーステストを書いてみる。

> 個々の測定値を受信（receive）し、それぞれの出現頻度を数えて集計するAggregatorクラスを扱っているとします。

### 実装

In [ ]:
class Aggregator:
    def __init__(self):
        self.__tally = {}

    def receive(self, value):
        self.__tally.setdefault(value, 0)
        self.__tally[value] += 1

    def tally(self):
        return self.__tally

### 事例ベーステスト

In [ ]:
%%ipytest

def test_tally():
    aggregator = Aggregator()
    aggregator.receive(1)
    aggregator.receive(2)
    aggregator.receive(3)
    aggregator.receive(2)
    tally = aggregator.tally()
    assert tally[1] == 1
    assert tally[2] == 2
    assert tally[3] == 1

### プロパティベーステスト

> Aggregatorのいくつかのプロパティを簡単な日本語で表現してみます。
> 
> - すべての測定値は、集計（tally）にキーとして出現する
> - 測定されなかった値は、集計に出現しない
> - 集計に出現した回数の合計は、受信した測定値の数と等しくなる
> - 測定の順序によって集計結果が変わることはない

In [ ]:
%%ipytest

from hypothesis import given
from hypothesis.strategies import integers, lists, permutations

@given(lists(integers()))
def test_all_measured_values_show_up_as_keys(measurements):
    """すべての測定値は、集計（tally）にキーとして出現する 
    """
    aggregator = Aggregator()
    for m in measurements:
        aggregator.receive(m)
    tally = aggregator.tally()
    assert set(measurements).issubset(tally.keys())

@given(lists(integers()))
def test_not_measured_values_do_not_show_up_as_keys(measurements):
    """測定されなかった値は、集計に出現しない 
       (対偶: 集計に出現した値は測定されている)
    """
    aggregator = Aggregator()
    for m in measurements:
        aggregator.receive(m)
    tally = aggregator.tally()
    assert set(tally.keys()).issubset(measurements)

@given(lists(integers()))
def test_sum_of_all_counts_is_number_of_measurements(measurements):
    """集計に出現した回数の合計は、受信した測定値の数と等しくなる
    """
    aggregator = Aggregator()
    for m in measurements:
        aggregator.receive(m)
    tally = aggregator.tally()
    assert sum(tally.values()) == len(measurements)

In [ ]:
%%ipytest

from hypothesis import assume, given
from hypothesis.strategies import composite, integers, lists, permutations

@composite
def list_and_permutation(draw, elements):
    xs = draw(lists(elements))
    permutation = draw(permutations(xs))
    assume(xs != permutation)
    return (xs, permutation)

@given(list_and_permutation(integers()))
def test_tally_does_not_depend_on_order_of_measurements(measurements_and_permutation):
    """測定の順序によって集計結果が変わることはない
    """
    (measurements, permutation) = measurements_and_permutation

    aggregator1 = Aggregator()
    for m in measurements:
        aggregator1.receive(m)
    tally1 = aggregator1.tally()

    aggregator2 = Aggregator()
    for m in measurements:
        aggregator2.receive(m)
    tally2 = aggregator2.tally()

    assert tally1 == tally2

In [ ]:
# 生成される値は example メソッドで確認できる
list_and_permutation(integers()).example()

`Aggregator` の実装を不具合のあるものに置き換えてテストを実行すると失敗の様子を観察できる。

# [Back to the Checkout](http://codekata.com/kata/kata09-back-to-the-checkout/) より

実践プロパティベーステストで例題として挙げられている[Back to the Checkout](http://codekata.com/kata/kata09-back-to-the-checkout/)について事例ベーステストとプロパティベーステストを書いてみる。

> この例題で参考としてる CodeKata の問題では、次のような表がサンプルとして与えられています。
>
> |Item|Unit Price|Special Price|
> |----|----------|-------------|
> | A  |       50 | 3 for 130   |
> | B  |        0 | 2 for  45   |
> | C  |       20 |             |
> | D  |       15 |             |
>
> 合計を出す関数は、商品が流れてくる順番によらず動作すべきです。したがって `["B","A","B"]` という順番で商品が流れてきたら、「2 つ買うと価格 45」の商品 B については特価の 45 が適用されることになるでしょう。このシステムでエクスポートすべき関数は、 `checkout:total(ListOfItemsBought, UnitPrices, SpecialPrices) → Price` だけです。
>
> <cite>[実践プロパティベーステスト &mdash; PropErとErlang/Elixrではじめよう](https://www.lambdanote.com/collections/proper-erlang-elixir), p.138</cite>

## 実装

In [ ]:
from collections import Counter

def checkout_total(
    list_of_items_bought: str, 
    unit_prices: dict[str, int], # item -> unit price
    special_prices: dict[str, (int, int)], # item -> (bundle size, bundle price)
):
    total = 0
    for item, number_of_items in Counter(list_of_items_bought).items():
        if item in special_prices:
            (bundle_size, bundle_price) = special_prices[item]
            total += (number_of_items // bundle_size) * bundle_price
            total += (number_of_items % bundle_size) * unit_prices[item]
        else:
            total += number_of_items * unit_prices[item]
    return total

## 事例ベーステスト

In [ ]:
%%ipytest

def test_checkout_total():
    unit_prices = {
        'A': 50,
        'B': 30,
        'C': 20,
        'D': 15,
    }
    special_prices = {
        'A': (3, 130),
        'B': (2, 45),
    }

    assert    0 == checkout_total(''      , unit_prices, special_prices)
    assert   50 == checkout_total('A'     , unit_prices, special_prices)
    assert   80 == checkout_total('AB'    , unit_prices, special_prices)
    assert  115 == checkout_total('CDBA'  , unit_prices, special_prices)

    assert  100 == checkout_total('AA'    , unit_prices, special_prices)
    assert  130 == checkout_total('AAA'   , unit_prices, special_prices)
    assert  180 == checkout_total('AAAA'  , unit_prices, special_prices)
    assert  230 == checkout_total('AAAAA' , unit_prices, special_prices)
    assert  260 == checkout_total('AAAAAA', unit_prices, special_prices)

    assert  160 == checkout_total('AAAB'  , unit_prices, special_prices)
    assert  175 == checkout_total('AAABB' , unit_prices, special_prices)
    assert  190 == checkout_total('DABABA', unit_prices, special_prices)

## プロパティベーステスト

In [ ]:
from hypothesis.strategies import characters, integers, lists, nothing, sampled_from, text

@composite
def checkout_total_patterns(draw):
    P = 1000
    N = 10
    
    # 単価表を生成する
    items = draw(lists(characters(min_codepoint=ord('A'), max_codepoint=ord('Z')), unique=True))
    prices = draw(lists(integers(min_value=1, max_value=P), min_size=len(items), max_size=len(items)))
    unit_prices = dict([ (item, price) for (item, price) in zip(items, prices) ])

    total = 0
    list_of_items_bought = ''

    # 特価表と特価商品の購入数および購入額を生成する
    special_prices = {}
    for item in draw(lists(sampled_from(items) if items else nothing(), unique=True)):
        bundle_size = draw(integers(min_value=2, max_value=N))
        bundle_price = draw(integers(min_value=1, max_value=bundle_size*unit_prices[item]))
        special_prices[item] = (bundle_size, bundle_price)
        # 特価が適用される分の購入数と購入額を生成する
        number_of_bundles = draw(integers(min_value=0, max_value=N))
        total += number_of_bundles * bundle_price
        list_of_items_bought += (number_of_bundles * bundle_size) * item
        # 特価が適用されない分の購入数と購入額を生成する
        number_of_rest_items = draw(integers(min_value=0, max_value=bundle_size-1))
        total += number_of_rest_items * unit_prices[item]
        list_of_items_bought += number_of_rest_items * item

    # 特価の設定されていない商品の購入数および購入額を生成する
    for item in items:
        if item not in special_prices:
            number_of_items = draw(integers(min_value=0, max_value=N))
            total += number_of_items * unit_prices[item]
            list_of_items_bought += number_of_items * item

    list_of_items_bought = ''.join(draw(permutations([*list_of_items_bought])))

    return total, list_of_items_bought, unit_prices, special_prices

In [ ]:
# 生成される値は example メソッドで確認できる
checkout_total_patterns().example()

In [ ]:
%%ipytest

from hypothesis import settings, Verbosity

@given(checkout_total_patterns())
# @settings(verbosity=Verbosity.debug)
def test_checkout_total(pattern):
    expected_total, list_of_items_bought, unit_prices, special_prices = pattern
    assert expected_total == checkout_total(list_of_items_bought, unit_prices, special_prices)    

`checkout_total` の実装を不具合のあるものに置き換えてテストを実行したり、その際に `verbosity` を変えたりしてみると、テスト失敗時の様子が分かる。

(例)

- 特別価格を無視する
- `//` を `/` に置き換える

## 標的型プロパティ

> Targeted property-based testing combines the advantages of both search-based and property-based testing. Instead of being completely random, T-PBT uses a search-based component to guide the input generation towards values that have a higher probability of falsifying a property. This explores the input space more effectively and requires fewer tests to find a bug or achieve a high confidence in the system being tested than random PBT. ([Löscher and Sagonas](http://proper.softlab.ntua.gr/Publications.html))
>
> <cite>[Targeted example generation](https://hypothesis.readthedocs.io/en/latest/details.html#targeted-example-generation)</cite>

> Example metrics:
> 
> - Number of elements in a collection, or tasks in a queue
> - Mean or maximum runtime of a task (or both, if you use label)
> - Compression ratio for data (perhaps per-algorithm or per-level)
> - Number of steps taken by a state machine
>
> [hypothesis.target(observation, *, label='')](https://hypothesis.readthedocs.io/en/latest/details.html#hypothesis.target)

In [ ]:
%%ipytest

from hypothesis import given, strategies as st, target

@given(st.floats(0, 1e100), st.floats(0, 1e100), st.floats(0, 1e100))
def test_associativity_with_target(a, b, c):
    ab_c = (a + b) + c
    a_bc = a + (b + c)
    difference = abs(ab_c - a_bc)
    target(difference)  # Without this, the test almost always passes
    assert difference < 2.0

## プロパティの生成

> Writing tests with Hypothesis frees you from the tedium of deciding on and writing out specific inputs to test. Now, the `hypothesis.extra.ghostwriter` module can write your test functions for you too!

https://hypothesis.readthedocs.io/en/latest/ghostwriter.html

In [ ]:
%pip install black

In [ ]:
from typing import Sequence
from hypothesis.extra import ghostwriter

def timsort(seq: Sequence[int]) -> Sequence[int]:
    return sorted(seq)

print('%%ipytest')
print(ghostwriter.idempotent(timsort))

# 参考資料など

- [実践プロパティベーステスト &mdash; PropErとErlang/Elixrではじめよう](https://www.lambdanote.com/collections/proper-erlang-elixir)
  - 「本格的なアプリケーション開発を通して、プロパティベーステストの基礎から極北までを体験できる、唯一無二の解説書です」
- [プロパティベース・テストを習得する](https://blogs.oracle.com/otnjp/post/know-for-sure-with-property-based-testing-ja)
  - 「PBTの最初の一歩を踏み出すとき、適切なプロパティを探すのは難しいタスクだと感じるかもしれません。プロパティの代表的な例に比べれば、現実世界でのプロパティの特定には別の種類の考え方が要求されます。プロパティを探す際に参考になる一連の便利なパターンが役立つかもしれません」
- [ユニットテストあれこれ ～ Haskeller の視点から ～](https://mew.org/~kazu/material/2018-unit-test.pdf)
  - 「突然変異がすべての性質をすり抜けたら性質が不足している」
  - 「たくさんの突然変異を作りそれぞれの性質を試す」
- [Misreading Chat](https://misreading.chat/)
  - [#97: Using Lightweight Formal Methods to Validate a Key-Value Storage Node in Amazon S3](https://misreading.chat/2022/08/22/97-using-lightweight-formal-methods-to-validate-a-key-value-storage-node-in-amazon-s3/)
    - 「簡単なモデルを静的に検証しつつ、簡単なモデルと本当の実装の一致はプロパティベースドテスティングで動的にチェックする」
- [Proptest Book](https://proptest-rs.github.io/proptest/intro.html)
  - [Differences between QuickCheck and Proptest](https://proptest-rs.github.io/proptest/proptest/vs-quickcheck.html)
    - "The one big difference is that QuickCheck generates and shrinks values based on type alone, whereas Proptest uses explicit `Strategy` objects. The QuickCheck approach has a lot of disadvantages in comparison"
- [Scala関数型デザイン＆プログラミング &mdash; Scalazコントリビューターによる関数型徹底ガイド](https://book.impress.co.jp/books/1114101091)
  - 第8章 プロパティベースのテスト
    - 「本章では、プロパティベースのテストを可能にするためのシンプルながら強力なライブラリに取り組みます」
- https://github.com/DRMacIver/minithesis
  - "This is is an incredibly minimal implementation of the core idea of Hypothesis, mostly intended for demonstration purposes to show someone how they might get the basics of it up and running."